In [1]:
#web based loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

#load, chunk and index the content of the html page

loader=WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post_content","post-header")
                     )))

text_documents=loader.load()

/Users/saaijeeshsn/Desktop/Langchain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
text_documents

[Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})]

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})]

In [8]:
import ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(model="mistral")
vectorstore = Chroma.from_documents(documents=documents,embedding=embeddings)

#create retriever
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in text_documents)

#define ollama llm function
def ollama_llm(question,context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='mistral',messages=[{'role':'user','content':formatted_prompt}])
    return response['message']['content']

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    return ollama_llm(question, formatted_context)

result=rag_chain("What is Task decomposition?")
print(result)




Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


 In the context of artificial intelligence and autonomous agents, task decomposition refers to the process of breaking down a complex problem or task into smaller, manageable subtasks. This approach allows for more efficient and effective solution strategies by enabling individual components to be solved independently and then combined to achieve the overall goal.

Task decomposition is an essential technique in many areas of AI research, including planning, scheduling, control, and multi-agent systems. It plays a crucial role in enabling intelligent agents to learn, reason, and act effectively in complex environments by allowing them to tackle problems incrementally and adapt their strategies based on the feedback from the environment or other agents.

Here's an example of task decomposition in a simple scenario: suppose we have a robot tasked with assembling a piece of furniture. The high-level goal is "assemble the furniture." However, this goal can be decomposed into several subgoa